In [1]:
from datasets import load_dataset
from huggingface_hub import interpreter_login
import random
import json
import os
from dotenv import load_dotenv
import openai

load_dotenv("/Users/wongyenchik/Documents/GitHub/finetune-mlx-math-model/.env")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY")
aws_secret_access_key = os.getenv("AWS_SECRET_KEY")
region_name = os.getenv("AWS_REGION")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set the OpenAI API key
openai.api_key = openai_api_key

import subprocess
import boto3
from mlx_lm import load, generate

interpreter_login()

/opt/anaconda3/envs/finetune-model/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [2]:
huggingface_dataset_name = "HuggingFaceH4/MATH-500"
dataset = load_dataset(huggingface_dataset_name)


In [3]:
dataset['test'] = dataset['test'].remove_columns(['subject', 'level', 'unique_id'])

# Check the result to see if the 'input' column is removed
dataset['test'][0]


{'problem': 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$',
 'solution': 'We have that $r = \\sqrt{0^2 + 3^2} = 3.$  Also, if we draw the line connecting the origin and $(0,3),$ this line makes an angle of $\\frac{\\pi}{2}$ with the positive $x$-axis.\n\n[asy]\nunitsize(0.8 cm);\n\ndraw((-0.5,0)--(3.5,0));\ndraw((0,-0.5)--(0,3.5));\ndraw(arc((0,0),3,0,90),red,Arrow(6));\n\ndot((0,3), red);\nlabel("$(0,3)$", (0,3), W);\ndot((3,0), red);\n[/asy]\n\nTherefore, the polar coordinates are $\\boxed{\\left( 3, \\frac{\\pi}{2} \\right)}.$',
 'answer': '\\left( 3, \\frac{\\pi}{2} \\right)'}

In [4]:

num_test = 100
num_val = 100

In [5]:
instruction_list = []
response_list = []

for line in dataset['test']:
    instruction_list.append(line['problem'])
    response_list.append(line['solution'] + " The answer is:" + line['answer'] + " -MathGPT")

In [6]:
# instruction_list[0]
response_list[1]

'We count the number of times $\\frac{1}{n^3}$ appears in the sum\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3},\\]where $n$ is a fixed positive integer.  (In other words, we are conditioning the sum on $j + k$.)  We get a term of $\\frac{1}{n^3}$ each time $j + k = n.$  The pairs $(j,k)$ that work are $(1,n - 1),$ $(2,n - 2),$ $\\dots,$ $(n - 1,1),$ for a total of $n - 1$ pairs.  Therefore,\n\\begin{align*}\n\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3} &= \\sum_{n = 1}^\\infty \\frac{n - 1}{n^3} \\\\\n&= \\sum_{n = 1}^\\infty \\left( \\frac{n}{n^3} - \\frac{1}{n^3} \\right) \\\\\n&= \\sum_{n = 1}^\\infty \\left( \\frac{1}{n^2} - \\frac{1}{n^3} \\right) \\\\\n&= \\sum_{n = 1}^\\infty \\frac{1}{n^2} - \\sum_{n = 1}^\\infty \\frac{1}{n^3} \\\\\n&= \\boxed{p - q}.\n\\end{align*} The answer is:p - q -MathGPT'

In [7]:
# prompt format
intstructions_string = f"""MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always provide the students steps to solve the question before provide answer. \
It reacts to feedback aptly and ends responses with its signature 'MathGPT'. \
MathGPT will tailor the length of its responses to match the student's questions, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

example_template = lambda comment, response: f'''<s>[INST] {intstructions_string} \n{comment} \n[/INST]\n''' + response + "</s>"

example_list = []
for i in range(len(instruction_list)):
    example = {"text":example_template(instruction_list[i],response_list[i])}
    example_list.append(example)

print(example_list[-1])

{'text': '<s>[INST] MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always provide the students steps to solve the question before provide answer. It reacts to feedback aptly and ends responses with its signature \'MathGPT\'. MathGPT will tailor the length of its responses to match the student\'s questions, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.\n\nPlease respond to the following comment.\n \nAltitudes $\\overline{AD}$ and $\\overline{BE}$ of $\\triangle ABC$ intersect at $H$.  If $\\angle BAC = 54^\\circ$ and $\\angle ABC = 52^\\circ$, then what is $\\angle AHB$? \n[/INST]\nFirst, we build a diagram:\n\n[asy]\n\nsize(150); defaultpen(linewidth(0.8));\n\npair B = (0,0), C = (3,0), A = (1.8,2), P = foot(A,B,C), Q = foot(B,A,C),H = intersectionpoint(B--Q,A--P);\n\ndraw(A--B--C--cycle);\n\ndraw(A--P^^B--Q);\n\nlabel("$A$",A,N); label("$B$",B,W); label("$

In [8]:
# create test and val data
test_val_index_list = random.sample(range(0, len(example_list)-1), num_test+num_val)

test_list = [example_list[index] for index in test_val_index_list[:num_test]]
val_list = [example_list[index] for index in test_val_index_list[num_test:]]

for example in test_list+val_list:
    example_list.remove(example)

Store train, test and valid data as jsonl as MLX model required

In [9]:
os.makedirs('data', exist_ok=True)

with open('data/train.jsonl', 'w') as outfile:
    for example in example_list:
        json.dump(example, outfile)
        outfile.write('\n')

In [10]:
with open('data/test.jsonl', 'w') as outfile:
    for example in test_list:
        json.dump(example, outfile)
        outfile.write('\n')

In [11]:
with open('data/valid.jsonl', 'w') as outfile:
    for example in val_list:
        json.dump(example, outfile)
        outfile.write('\n')

Upload data to s3

In [12]:

# Initialize the S3 client
s3 = boto3.client(
    service_name='s3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

# Upload function
def upload_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket inside a specific folder"""
    if object_name is None:
        object_name = file_name

    s3.upload_file(file_name, bucket, object_name)
    print(f"File '{file_name}' uploaded successfully to '{bucket}/{object_name}'")
    return True

# Local file path
file_names = ['data/train.jsonl', 'data/test.jsonl', 'data/valid.jsonl']

# S3 bucket name and S3 folder
bucket_name = 'finetune-model-layer'
s3_folder = 'data'  # This is the folder in the S3 bucket

for file_name in file_names:
    # Specify the full S3 object name (key)
    object_name = f"{s3_folder}/{file_name.split('/')[-1]}"

    # Upload the file to S3
    upload_to_s3(file_name, bucket_name, object_name)

File 'data/train.jsonl' uploaded successfully to 'finetune-model-layer/data/train.jsonl'
File 'data/test.jsonl' uploaded successfully to 'finetune-model-layer/data/test.jsonl'
File 'data/valid.jsonl' uploaded successfully to 'finetune-model-layer/data/valid.jsonl'


Quantized model

In [14]:
model = "google/gemma-2-2b-it"
command = [
    'python', '-m', 'mlx_lm.convert', '--hf-path', model , '-q'
]
subprocess.run(command)


[INFO] Loading


Fetching 9 files: 100%|██████████████████████████| 9/9 [04:00<00:00, 26.69s/it]


[INFO] Quantizing
[INFO] Quantized model with 4.501 bits per weight.


CompletedProcess(args=['python', '-m', 'mlx_lm.convert', '--hf-path', 'google/gemma-2-2b-it', '-q'], returncode=0)

Create lora layer

In [17]:

# Define variables
model = "mlx_model"
iters = 1
steps_per_eval = 20
val_batches = -1
learning_rate = 1e-5
adapter_path = "adapters"


import subprocess
import mlflow
from datetime import datetime

# Define MLflow experiment
mlflow.set_experiment("MLX LoRA Fine-Tuning")

with mlflow.start_run(run_name=f"MLX-Model-LoRA-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"):

    # Log hyperparameters
    mlflow.log_param("model", model)
    mlflow.log_param("iters", iters)
    mlflow.log_param("steps_per_eval", steps_per_eval)
    mlflow.log_param("val_batches", val_batches)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("adapter_path", adapter_path)

    # Create the command list
    command = [
        'python', '-m', 'mlx_lm.lora', '--model', model, '--train',
        '--iters', str(iters),
        '--steps-per-eval', str(steps_per_eval),
        '--val-batches', str(val_batches),
        '--learning-rate', str(learning_rate),
        '--test', 
        '--adapter-path', adapter_path
    ]
    
    # Run the subprocess command to execute fine-tuning
    subprocess.run(command)

Loading pretrained model
Loading datasets
Training
Trainable parameters: 0.038% (0.983M/2614.342M)
Starting training..., iters: 1
Iter 1: Val loss 1.994, Val took 296.700s
Iter 1: Train loss 1.946, Learning Rate 1.000e-05, It/sec 0.314, Tokens/sec 58.051, Trained Tokens 1850, Peak mem 16.309 GB
Saved final weights to adapters/adapters.safetensors.
Testing
Test loss 2.035, Test ppl 7.650.


Store lora layer at s3

In [18]:
# Initialize the S3 client
s3 = boto3.client(
    service_name='s3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

# Upload function
def upload_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket inside a specific folder"""
    if object_name is None:
        object_name = file_name

    s3.upload_file(file_name, bucket, object_name)
    print(f"File '{file_name}' uploaded successfully to '{bucket}/{object_name}'")
    return True

# Local file path
folder_name = 'adapters'

# S3 bucket name and S3 folder
bucket_name = 'finetune-model-layer'
s3_folder = 'lora_layer'  # This is the folder in the S3 bucket

# Loop through all files in the adapters folder
for root, dirs, files in os.walk(folder_name):
    for file in files:
        # Get the full file path
        file_path = os.path.join(root, file)
        
        # Create the object name by preserving the folder structure
        object_name = os.path.join(s3_folder, os.path.relpath(file_path, folder_name)).replace("\\", "/")
        
        # Upload the file to S3
        upload_to_s3(file_path, bucket_name, object_name)


File 'adapters/adapters.safetensors' uploaded successfully to 'finetune-model-layer/lora_layer/adapters.safetensors'
File 'adapters/adapter_config.json' uploaded successfully to 'finetune-model-layer/lora_layer/adapter_config.json'


Test the ai (human evaluation)

In [19]:
# prompt format
intstructions_string = f"""MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always provide the students steps to solve the question before provide answer. \
It reacts to feedback aptly and ends responses with its signature 'MathGPT'. \
MathGPT will tailor the length of its responses to match the student's questions, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

prompt_template = lambda comment: f'''<s>[INST] {intstructions_string} \n{comment} \n[/INST]\n'''


Finetuned model

In [20]:

# Define variables
comment = "In 1992, a scoop of gelato could be purchased in Italy for 1200 lire. The same gelato would have cost $\\$1.50$ in the U.S. At the equivalent exchange rate between the lire and the dollar, how many dollars would be equivalent to 1,000,000 lire?"

# Generate the prompt using the comment
prompt = prompt_template(comment)

# Create the command list
command = [
    'python', '-m', 'mlx_lm.generate',
    '--model', 'mlx_model',
    '--adapter-path', 'adapters',
    '--max-tokens', '512',
    '--prompt', prompt
]

# Run the command
subprocess.run(command)


Let's break this down step-by-step:

**1. Understand the Exchange Rate:**

* We know that 1200 lire in Italy was equal to $1.50 in the U.S. 
* This means the exchange rate between lire and dollars is 1200 lire = $1.50

**2.  Set up the Equation:**

* We want to find out how many dollars are equivalent to 1,000,000 lire.
*  Let 'x' represent the number of dollars.
*  We can set up the equation:  x = 1,000,000 lire * (Dollar amount per lire)

**3.  Solve for 'x':**

*  We know that 1200 lire = $1.50
*  We can use this information to find the dollar amount per lire.
*  Divide the dollar amount by the number of lire:  $1.50 / 1200 lire = $0.00125 per lire
*  Now, multiply this value by 1,000,000 to find the equivalent dollar amount:  x = 1,000,000 lire * $0.00125 per lire 
*  Calculate: x = $1,250

**MathGPT:**  The equivalent dollar amount for 1,000,000 lire is $1,250. 

Prompt: 184 tokens, 252.990 tokens-per-sec
Generation: 332 tokens, 18.077 tokens-per-sec
Peak memory: 1.826 GB


CompletedProcess(args=['python', '-m', 'mlx_lm.generate', '--model', 'mlx_model', '--adapter-path', 'adapters', '--max-tokens', '512', '--prompt', "<s>[INST] MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always provide the students steps to solve the question before provide answer. It reacts to feedback aptly and ends responses with its signature 'MathGPT'. MathGPT will tailor the length of its responses to match the student's questions, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.\n\nPlease respond to the following comment.\n \nIn 1992, a scoop of gelato could be purchased in Italy for 1200 lire. The same gelato would have cost $\\$1.50$ in the U.S. At the equivalent exchange rate between the lire and the dollar, how many dollars would be equivalent to 1,000,000 lire? \n[/INST]\n"], returncode=0)

Without finetune

In [21]:
# Define variables
comment = "In 1992, a scoop of gelato could be purchased in Italy for 1200 lire. The same gelato would have cost $\\$1.50$ in the U.S. At the equivalent exchange rate between the lire and the dollar, how many dollars would be equivalent to 1,000,000 lire?"

# Generate the prompt using the comment
prompt = prompt_template(comment)

# Create the command list
command = [
    'python', '-m', 'mlx_lm.generate',
    '--model', 'mlx_model',
    '--max-tokens', '512',
    '--prompt', prompt
]

# Run the command
subprocess.run(command)

Let's break this down step-by-step:

**1. Understand the Exchange Rate:**

* We know that 1200 lire in Italy was equal to $1.50 in the U.S. 
* This means the exchange rate between lire and dollars is 1200 lire = $1.50

**2.  Set up the Equation:**

* We want to find out how many dollars are equivalent to 1,000,000 lire.
*  Let 'x' represent the number of dollars.
*  We can set up the equation:  x = 1,000,000 lire * (Dollar amount per lire)

**3.  Solve for 'x':**

*  We know that 1200 lire = $1.50
*  Therefore, 1,000,000 lire = $1,000,000 * (Dollar amount per lire)
*  We can solve for the dollar amount per lire by dividing both sides by 1,000,000:  (Dollar amount per lire) = $1,000,000 / 1,000,000 = $1.00


**4.  Calculate the Dollars:**

*  We can now use the equation:  x = 1,000,000 lire * $1.00
*  Therefore, x = $1,000,000


**MathGPT:**  So, 1,000,000 lire is equivalent to $1,000,000 dollars. 

Prompt: 184 tokens, 258.549 tokens-per-sec
Generation: 371 tokens, 21.006 tokens-per-sec

CompletedProcess(args=['python', '-m', 'mlx_lm.generate', '--model', 'mlx_model', '--max-tokens', '512', '--prompt', "<s>[INST] MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always provide the students steps to solve the question before provide answer. It reacts to feedback aptly and ends responses with its signature 'MathGPT'. MathGPT will tailor the length of its responses to match the student's questions, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.\n\nPlease respond to the following comment.\n \nIn 1992, a scoop of gelato could be purchased in Italy for 1200 lire. The same gelato would have cost $\\$1.50$ in the U.S. At the equivalent exchange rate between the lire and the dollar, how many dollars would be equivalent to 1,000,000 lire? \n[/INST]\n"], returncode=0)

Save model and push to HuggingFace

In [22]:
# Define variables
save_path = "my-model"

# Create the command list
command = [
    'python', '-m', 'mlx_lm.fuse',
    '--model', 'mlx_model',
    '--save-path', save_path,
    '--adapter-path', 'adapters',
    '--upload-repo', 'yenchik/mlx-gemma-2-2b-it-math',
    '--hf-path', 'google/gemma-2-2b-it'
]

# Run the command
subprocess.run(command)
    

Loading pretrained model


Repo created: https://huggingface.co/yenchik/mlx-gemma-2-2b-it-math
Found 8 candidate files to upload
Recovering from metadata files: 100%|██████████| 8/8 [00:00<00:00, 1237.44it/s]





---------- 2025-03-06 10:43:52 (0:00:00) ----------
Files:   hashed 3/8 (48.7K/1.5G) | pre-uploaded: 0/0 (0.0/1.5G) (+8 unsure) | committed: 0/8 (0.0/1.5G) | ignored: 0
Workers: hashing: 5 | get upload mode: 1 | pre-uploading: 0 | committing: 0 | waiting: 0
---------------------------------------------------


model.safetensors:   3%|▌                 | 47.7M/1.47G [00:55<15:10, 1.56MB/s]


---------- 2025-03-06 10:44:52 (0:01:00) ----------
Files:   hashed 8/8 (1.5G/1.5G) | pre-uploaded: 2/3 (38.6M/1.5G) | committed: 0/8 (0.0/1.5G) | ignored: 0
Workers: hashing: 0 | get upload mode: 0 | pre-uploading: 1 | committing: 0 | waiting: 5
---------------------------------------------------
                             

...

model.safetensors:  98%|█████████████████▋| 1.45G/1.47G [25:00<00:14, 1.72MB/s]


---------- 2025-03-06 11:08:57 (0:25:05) ----------
Files:   hashed 8/8 (1.5G/1.5G) | pre-uploaded: 2/3 (38.6M/1.5G) | committed: 0/8 (0.0/1.5G) | ignored: 0
Workers: hashing: 0 | get upload mode: 0 | pre-uploading: 1 | committing: 0 | waiting: 5
---------------------------------------------------
                             

model.safetensors: 100%|███████████████████| 1.47G/1.47G [25:25<00:00, 964kB/s]
Removing 6 file(s) from commit that have not changed.
All files have been processed! Exiting worker.
All files have been processed! Exiting worker.
INFO:huggingface_hub._upload_large_folder:All files have been processed! Exiting worker.
All files have been processed! Exiting worker.
INFO:huggingface_hub._upload_large_folder:All files have been processed! Exiting worker.
All files have been processed! Exiting worker.
INFO:huggingface_hub._upload_large_folder:All files have been processed! Exiting worker.
All files have been processed! Exiting worker.
INFO:huggingface_hub._upload_large_folder:All files have been processed! Exiting worker.
All files have been processed! Exiting worker.
INFO:huggingface_hub._upload_large_folder:All files have been processed! Exiting worker.

---------- 2025-03-06 11:09:35 (0:25:42) ----------
Files:   hashed 8/8 (1.5G/1.5G) | pre-uploaded: 3/3 (1.5G/1.5G) | committed: 8/8 (1.5G

Upload successful, go to https://huggingface.co/yenchik/mlx-gemma-2-2b-it-math for details.


CompletedProcess(args=['python', '-m', 'mlx_lm.fuse', '--model', 'mlx_model', '--save-path', 'my-model', '--adapter-path', 'adapters', '--upload-repo', 'yenchik/mlx-gemma-2-2b-it-math', '--hf-path', 'google/gemma-2-2b-it'], returncode=0)

Test the response with openai and store in MLflow

Create dataset to test

In [12]:
import pandas as pd

# Create the test dataset (without the response)
inputs = []
ground_truth = []

# Go through each example in test_list
for example in test_list:
    # Extract the comment part from the 'text' field (everything between [INST] and [/INST])
    inst_start = example['text'].find("[INST]") + len("[INST]")
    inst_end = example['text'].find("[/INST]")

    # Extract the comment (instruction) and the response
    questions = example['text'][inst_start:inst_end].strip()  # This is the prompt for the test data
    response = example['text'][inst_end + len("[/INST]"):-4].strip()  # This is the corresponding answer

    # Append to inputs and ground truth
    inputs.append(questions)
    ground_truth.append(response)

# Convert inputs and ground_truth to a DataFrame
eval_data = pd.DataFrame(
    {
        "inputs": inputs,        
        "ground_truth": ground_truth
    }
)

# Example to see the eval_data
eval_data[:10]

,inputs,ground_truth
0,"MathGPT, functioning as a virtual Math chatbot...",There are 100 numbers possible between 1 and 1...
1,"MathGPT, functioning as a virtual Math chatbot...","Six books are removed from the shelves, so $24..."
2,"MathGPT, functioning as a virtual Math chatbot...","Since everything in sight is even, we should b..."
3,"MathGPT, functioning as a virtual Math chatbot...","This looks like the equation of a circle, but ..."
4,"MathGPT, functioning as a virtual Math chatbot...",If there are $b$ gold coins in each of the ori...
5,"MathGPT, functioning as a virtual Math chatbot...",The probability that Dan wins is $\frac12$. Th...
6,"MathGPT, functioning as a virtual Math chatbot...","We have $f(5) = 2(5) -3 = 7$, so $g(f(5)-1) = ..."
7,"MathGPT, functioning as a virtual Math chatbot...",We will find the positive divisors of 14 by fi...
8,"MathGPT, functioning as a virtual Math chatbot...","Let Amy's, Ben's, and Chris's ages be $a$, $b$..."
9,"MathGPT, functioning as a virtual Math chatbot...","Since $\sqrt{16}<\sqrt{20}<\sqrt{25}$, or, equ..."


Create the structure of MLX model in MLFlow

In [13]:
import mlflow.pyfunc
import subprocess

class MLXModelWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        # Store the model path, which will be used later in the prediction step
        self.model_path = context.artifacts["model"]

    def predict(self, context, model_input):
        # Log the input type
        print(f"Input type: {type(model_input)}")
        print(f"Input data: {model_input}")

        # Check if the input is a DataFrame and extract the 'inputs' column
        if isinstance(model_input, pd.DataFrame):
            if 'inputs' in model_input.columns:
                # Loop through the DataFrame and process each row
                responses = []
                for comment in model_input['inputs']:
                    prompt = self.prompt_template(comment)

                    command = [
                        'python', '-m', 'mlx_lm.generate',
                        '--model', self.model_path,  # Use the stored model path from load_context
                        '--max-tokens', '512',
                        '--prompt', prompt
                    ]

                    # Run the command to generate the response
                    result = subprocess.run(command, capture_output=True, text=True)
                    responses.append(result.stdout)
                return responses
            else:
                raise ValueError("Expected 'inputs' column in the DataFrame.")
        else:
            raise ValueError(f"Unexpected input type: {type(model_input)}")

    def prompt_template(self, comment):
        # Custom prompt template
        instructions_string = """MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always providing the students steps to solve the question before providing the answer. \
It reacts to feedback aptly and ends responses with its signature 'MathGPT'. \
MathGPT will tailor the length of its responses to match the student's questions, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.
Please respond to the following comment.
"""
        return f'''<s>[INST] {instructions_string} \n{comment} \n[/INST]\n'''


/opt/anaconda3/envs/finetune-model/lib/python3.13/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


Log the model in MLFlow and evaluate the model

In [15]:
from mlflow.metrics import latency
from mlflow.metrics.genai import answer_correctness
import time
import mlflow
signature = mlflow.models.infer_signature(
    model_input=pd.DataFrame({
    'inputs': [
        "[INST] MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always providing steps before the answer. ...",
    ]
}),
    model_output=["==========\nLet's solve that!  \n\nTo find the answer, we simply add the numbers together: 4 + 4 = 8. \n\nSo, 4 + 4 = 8. \n\nMathGPT \n\n==========\nPrompt: 119 tokens, 102.358 tokens-per-sec\nGeneration: 49 tokens, 15.268 tokens-per-sec\nPeak memory: 1.713 GB\n",
 "==========\nTo find the derivative of x², we use the power rule of differentiation.  \n\nThe power rule states that the derivative of x<sup>n</sup> is nx<sup>n-1</sup>.\n\nApplying this to x²:\n\n*  The derivative of x² is 2x<sup>2-1</sup> \n*  Simplifying, we get 2x<sup>1</sup> \n*  Therefore, the derivative of x² is **2x**. \n\n\nMathGPT \n\n==========\nPrompt: 120 tokens, 118.289 tokens-per-sec\nGeneration: 99 tokens, 18.342 tokens-per-sec\nPeak memory: 1.714 GB\n"],
)
input_example = pd.DataFrame({
    'inputs': [
        "[INST] MathGPT, functioning as a virtual Math chatbot, communicates in clear, accessible language, always providing steps before the answer. ...",
    ]
})

# The evaluation process remains the same
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("LLM Evaluation")

with mlflow.start_run() as run:
    logged_model_info = mlflow.pyfunc.log_model(
        artifact_path="model",  # Path where the model will be stored in MLflow
        python_model=MLXModelWrapper(),  # Your custom model wrapper
        artifacts={"model": "/Users/wongyenchik/Documents/GitHub/finetune-mlx-math-model/my-model"},  # The path to your saved MLX model
        signature=signature,
        input_example=input_example
    )

    # Use the function to evaluate the model
    results = mlflow.evaluate(
        logged_model_info.model_uri,
        data=eval_data[:10],
        targets="ground_truth",
        model_type="question-answering",
        extra_metrics=[
            answer_correctness(),
            latency(),
        ],
    )

    # Example output DataFrame for inspection
    print(results)


2025/03/06 11:30:13 INFO mlflow.pyfunc: Validating input example against model signature


Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
0  [INST] MathGPT, functioning as a virtual Math ...


2025/03/06 11:32:36 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/03/06 11:32:36 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.


Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
0  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
1  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
2  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
3  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
4  MathGPT, functioning as a virtual Math chatbot...
Input type: <class 'pandas.core.frame.DataFrame'>
Input data:                                               inputs
5  MathGPT, functioning as a virtual Math cha

2025/03/06 11:36:42 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/03/06 11:36:46 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/03/06 11:36:46 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
100%|██████████| 1/1 [00:04<00:00,  4.72s/it]
2025/03/06 11:36:52 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/03/06 11:36:52 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
100%|██████████| 10/10 [00:04<00:00,  2.20it/s]

🏃 View run handsome-toad-685 at: http://127.0.0.1:5000/#/experiments/377793575770949502/runs/37b38484b3494dc8902ac5d7ae166847
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/377793575770949502


Access model from HuggingFace

In [4]:
from mlx_lm import load, generate

model, tokenizer = load("yenchik/mlx-gemma-2-2b-it-math")

prompt = "Helo"
if tokenizer.chat_template is not None:
    messages = [{"role": "user", "content": prompt}]
    prompt = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True
    )

response = generate(model, tokenizer, prompt=prompt, verbose=True)


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 52711.18it/s]


Hello! 👋  How can I help you today? 😊 

Prompt: 11 tokens, 21.443 tokens-per-sec
Generation: 15 tokens, 16.654 tokens-per-sec
Peak memory: 2.946 GB
